# Getting Started with UCC: Optimizing Circuits for Fake Devices

This notebook demonstrates how to use the Unitary Compiler Collection (UCC) to optimize quantum circuits for specific hardware backends, including fake devices. We'll show how UCC can significantly improve circuit performance by reducing gate counts and depth.

## What is UCC?

UCC is a Python library for frontend-agnostic, high-performance compilation of quantum circuits. It supports multiple quantum computing frameworks and can optimize circuits for specific hardware targets.

## Installation

If you haven't installed UCC yet, you can do so with:

```bash
pip install ucc
```

For development or to access all features:

```bash
git clone https://github.com/unitaryfoundation/ucc.git
cd ucc
uv sync --all-extras --all-groups
```

In [ ]:
# Import required libraries
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit.providers.fake_provider import GenericBackendV2
from qiskit.circuit.random import random_circuit
from qiskit.transpiler import Target
import matplotlib.pyplot as plt

# Import UCC
import ucc

print(f"UCC version: {ucc.__version__}")
print(f"Supported formats: {ucc.supported_circuit_formats}")

## Creating a Test Circuit

Let's create a random quantum circuit to demonstrate UCC's optimization capabilities.

In [ ]:
# Create a random circuit for testing
num_qubits = 5
depth = 10
seed = 42

original_circuit = random_circuit(
    num_qubits=num_qubits,
    depth=depth,
    seed=seed,
    max_operands=2
)

print(f"Original circuit depth: {original_circuit.depth()}")
print(f"Original circuit size: {original_circuit.size()}")
print(f"Original circuit gates: {original_circuit.count_ops()}")

# Visualize the original circuit
print("\nOriginal Circuit:")
print(original_circuit.draw(output='text', fold=-1))

## Basic UCC Compilation

Let's compile this circuit using UCC's default optimization passes.

In [ ]:
# Compile with UCC using default settings
compiled_circuit = ucc.compile(original_circuit)

print(f"UCC compiled circuit depth: {compiled_circuit.depth()}")
print(f"UCC compiled circuit size: {compiled_circuit.size()}")
print(f"UCC compiled circuit gates: {compiled_circuit.count_ops()}")

# Calculate improvement
depth_improvement = (original_circuit.depth() - compiled_circuit.depth()) / original_circuit.depth() * 100
size_improvement = (original_circuit.size() - compiled_circuit.size()) / original_circuit.size() * 100

print(f"\nImprovements:")
print(f"Depth reduction: {depth_improvement:.1f}%")
print(f"Size reduction: {size_improvement:.1f}%")

print("\nUCC Compiled Circuit:")
print(compiled_circuit.draw(output='text', fold=-1))

## Optimizing for Fake Hardware Devices

Now let's demonstrate UCC's capabilities with fake hardware devices. We'll use Qiskit's fake provider to simulate real hardware constraints.

In [ ]:
# Set up fake devices using GenericBackendV2
from qiskit.providers.models import BackendProperties
from qiskit.transpiler import CouplingMap

# Create mock backends with different topologies
def create_mock_backend(num_qubits, coupling_map, name):
    """Create a mock backend for testing."""
    return GenericBackendV2(
        num_qubits=num_qubits,
        basis_gates=['cx', 'rz', 'ry', 'rx', 'h', 's', 't'],
        coupling_map=coupling_map,
        seed=42
    )

# Different coupling topologies
linear_coupling = CouplingMap([(i, i+1) for i in range(4)])
grid_coupling = CouplingMap([(0,1), (1,2), (2,3), (0,3), (1,4)])
all_to_all_coupling = None  # Fully connected

fake_devices = {
    'Linear5Q': create_mock_backend(5, linear_coupling, 'Linear5Q'),
    'Grid5Q': create_mock_backend(5, grid_coupling, 'Grid5Q'),
    'AllToAll5Q': create_mock_backend(5, all_to_all_coupling, 'AllToAll5Q')
}

# Compare performance across different fake devices
results = {}

for device_name, fake_device in fake_devices.items():
    print(f"\n=== Optimizing for {device_name} ===")
    
    # Get device properties
    coupling_map = fake_device.coupling_map
    basis_gates = fake_device.operation_names
    
    print(f"Coupling map: {coupling_map}")
    print(f"Basis gates: {basis_gates}")
    
    # Compile with UCC for this specific device
    device_compiled = ucc.compile(
        original_circuit,
        target_device=fake_device
    )
    
    # Also compile with Qiskit's default transpiler for comparison
    qiskit_compiled = transpile(
        original_circuit,
        backend=fake_device,
        optimization_level=3
    )
    
    results[device_name] = {
        'ucc_depth': device_compiled.depth(),
        'ucc_size': device_compiled.size(),
        'qiskit_depth': qiskit_compiled.depth(),
        'qiskit_size': qiskit_compiled.size(),
        'original_depth': original_circuit.depth(),
        'original_size': original_circuit.size()
    }
    
    print(f"UCC - Depth: {device_compiled.depth()}, Size: {device_compiled.size()}")
    print(f"Qiskit - Depth: {qiskit_compiled.depth()}, Size: {qiskit_compiled.size()}")
    
    # Calculate improvements
    ucc_depth_imp = (original_circuit.depth() - device_compiled.depth()) / original_circuit.depth() * 100
    qiskit_depth_imp = (original_circuit.depth() - qiskit_compiled.depth()) / original_circuit.depth() * 100
    
    print(f"UCC depth improvement: {ucc_depth_imp:.1f}%")
    print(f"Qiskit depth improvement: {qiskit_depth_imp:.1f}%")

## Performance Comparison Visualization

Let's visualize the performance improvements across different devices.

In [ ]:
# Create visualization
devices = list(results.keys())
ucc_depths = [results[d]['ucc_depth'] for d in devices]
qiskit_depths = [results[d]['qiskit_depth'] for d in devices]
original_depth = results[devices[0]]['original_depth']

# Set up the plot
x = np.arange(len(devices))
width = 0.35

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Depth comparison
bars1 = ax1.bar(x - width/2, ucc_depths, width, label='UCC Compiled', color='skyblue')
bars2 = ax1.bar(x + width/2, qiskit_depths, width, label='Qiskit Compiled', color='lightcoral')
ax1.axhline(y=original_depth, color='red', linestyle='--', label='Original Circuit')

ax1.set_xlabel('Fake Device')
ax1.set_ylabel('Circuit Depth')
ax1.set_title('Circuit Depth Comparison')
ax1.set_xticks(x)
ax1.set_xticklabels(devices)
ax1.legend()

# Add value labels on bars
for bar in bars1:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
             f'{int(height)}', ha='center', va='bottom')
    
for bar in bars2:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
             f'{int(height)}', ha='center', va='bottom')

# Size comparison
ucc_sizes = [results[d]['ucc_size'] for d in devices]
qiskit_sizes = [results[d]['qiskit_size'] for d in devices]
original_size = results[devices[0]]['original_size']

bars3 = ax2.bar(x - width/2, ucc_sizes, width, label='UCC Compiled', color='skyblue')
bars4 = ax2.bar(x + width/2, qiskit_sizes, width, label='Qiskit Compiled', color='lightcoral')
ax2.axhline(y=original_size, color='red', linestyle='--', label='Original Circuit')

ax2.set_xlabel('Fake Device')
ax2.set_ylabel('Circuit Size (Total Gates)')
ax2.set_title('Circuit Size Comparison')
ax2.set_xticks(x)
ax2.set_xticklabels(devices)
ax2.legend()

# Add value labels on bars
for bar in bars3:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
             f'{int(height)}', ha='center', va='bottom')
    
for bar in bars4:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
             f'{int(height)}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

# Print summary statistics
print("\n=== Performance Summary ===")
for device in devices:
    ucc_depth = results[device]['ucc_depth']
    qiskit_depth = results[device]['qiskit_depth']
    original_depth = results[device]['original_depth']
    
    ucc_depth_pct = (original_depth - ucc_depth) / original_depth * 100
    qiskit_depth_pct = (original_depth - qiskit_depth) / original_depth * 100
    
    print(f"{device}:")
    print(f"  UCC depth improvement: {ucc_depth_pct:.1f}%")
    print(f"  Qiskit depth improvement: {qiskit_depth_pct:.1f}%")
    print(f"  UCC better by: {qiskit_depth_pct - ucc_depth_pct:.1f} percentage points")

## Advanced UCC Features

Let's explore some advanced UCC features like custom passes and different target formats.

In [ ]:
# Demonstrate different output formats
print("=== Different Output Formats ===")

# Compile to different formats
qiskit_result = ucc.compile(original_circuit, return_format="qiskit")
print(f"Qiskit format: {type(qiskit_result)}")

# Try Cirq if available
try:
    cirq_result = ucc.compile(original_circuit, return_format="cirq")
    print(f"Cirq format: {type(cirq_result)}")
except Exception as e:
    print(f"Cirq conversion failed: {e}")

# Demonstrate custom gate sets
print("\n=== Custom Gate Set Compilation ===")
custom_gateset = {"cx", "rz", "ry", "rx", "h"}
custom_compiled = ucc.compile(
    original_circuit,
    target_gateset=custom_gateset
)

print(f"Custom gateset compilation:")
print(f"Depth: {custom_compiled.depth()}")
print(f"Size: {custom_compiled.size()}")
print(f"Gate counts: {custom_compiled.count_ops()}")

## Custom Pass Example

Let's create a simple custom pass to demonstrate UCC's extensibility.

In [ ]:
# Create a simple custom pass that counts gates
from qiskit.transpiler.basepasses import TransformationPass
from qiskit.dagcircuit import DAGCircuit

class GateCounterPass(TransformationPass):
    """A simple pass that counts and reports gate statistics."""
    
    def __init__(self):
        super().__init__()
        self.gate_counts = {}
    
    def run(self, dag: DAGCircuit) -> DAGCircuit:
        # Count gates in the DAG
        for node in dag.op_nodes():
            gate_name = node.op.name
            if gate_name in self.gate_counts:
                self.gate_counts[gate_name] += 1
            else:
                self.gate_counts[gate_name] = 1
        
        print(f"Gate counts in circuit: {self.gate_counts}")
        return dag

# Use the custom pass
print("=== Using Custom Pass ===")
gate_counter = GateCounterPass()
circuit_with_custom_pass = ucc.compile(
    original_circuit,
    custom_passes=[gate_counter]
)

print(f"Circuit after custom pass - Depth: {circuit_with_custom_pass.depth()}, Size: {circuit_with_custom_pass.size()}")

## Real-World Example: Quantum Fourier Transform

Let's demonstrate UCC's capabilities with a more realistic quantum algorithm - the Quantum Fourier Transform (QFT).

In [ ]:
# Create a QFT circuit
def qft_circuit(n_qubits):
    """Create a Quantum Fourier Transform circuit."""
    qc = QuantumCircuit(n_qubits)
    
    for i in range(n_qubits):
        qc.h(i)
        for j in range(i+1, n_qubits):
            qc.cp(np.pi / (2**(j-i)), j, i)
    
    # Add swaps for inverse QFT
    for i in range(n_qubits//2):
        qc.swap(i, n_qubits-i-1)
    
    return qc

# Create QFT circuit
qft_size = 4
qft_original = qft_circuit(qft_size)

print(f"QFT Circuit (n={qft_size}):")
print(f"Original depth: {qft_original.depth()}")
print(f"Original size: {qft_original.size()}")

# Compile with UCC
qft_compiled = ucc.compile(qft_original)

print(f"UCC compiled depth: {qft_compiled.depth()}")
print(f"UCC compiled size: {qft_compiled.size()}")

# Compile for a specific fake device
qft_device_compiled = ucc.compile(qft_original, target_device=fake_devices['Linear5Q'])

print(f"UCC + Linear5Q depth: {qft_device_compiled.depth()}")
print(f"UCC + Linear5Q size: {qft_device_compiled.size()}")

# Compare with Qiskit transpilation
qft_qiskit = transpile(qft_original, backend=fake_devices['Linear5Q'], optimization_level=3)
print(f"Qiskit + Linear5Q depth: {qft_qiskit.depth()}")
print(f"Qiskit + Linear5Q size: {qft_qiskit.size()}")

print("\nQFT Circuit Visualization:")
print(qft_compiled.draw(output='text', fold=-1))

## Summary

In this notebook, we've demonstrated:

1. **Basic UCC Usage**: How to compile circuits with UCC's default optimization passes
2. **Hardware-Aware Compilation**: Optimizing circuits for specific fake devices
3. **Performance Comparison**: Comparing UCC's performance against Qiskit's built-in transpiler
4. **Advanced Features**: Custom passes, different output formats, and custom gate sets
5. **Real-World Example**: Optimizing a Quantum Fourier Transform circuit

### Key Takeaways:

- UCC can significantly reduce circuit depth and gate count
- Hardware-aware compilation is crucial for real device performance
- UCC's modular design allows for easy integration of custom optimization passes
- The library supports multiple quantum computing frameworks

### Next Steps:

- Explore UCC's built-in passes like `BQSKitTransformationPass` and `MPSPass`
- Try compiling circuits for real quantum hardware backends
- Develop custom optimization passes for specific algorithms
- Benchmark UCC against other quantum compilers

For more information, visit the [UCC documentation](https://ucc.readthedocs.io/) or join the discussion on [GitHub](https://github.com/unitaryfoundation/ucc).